In [0]:
%fs ls dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet

path,name,size,modificationTime
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/_SUCCESS,_SUCCESS,0,1702997591000
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/_committed_8263679197730626570,_committed_8263679197730626570,424,1702997591000
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/_started_8263679197730626570,_started_8263679197730626570,0,1702997583000
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/part-00000-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-298-1-c000.snappy.parquet,part-00000-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-298-1-c000.snappy.parquet,45030594,1702997590000
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/part-00001-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-299-1-c000.snappy.parquet,part-00001-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-299-1-c000.snappy.parquet,45628171,1702997590000
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/part-00002-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-300-1-c000.snappy.parquet,part-00002-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-300-1-c000.snappy.parquet,45789829,1702997590000
dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet/part-00003-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-301-1-c000.snappy.parquet,part-00003-tid-8263679197730626570-9114dca2-1586-46f5-bdfe-ae103061bb2b-301-1-c000.snappy.parquet,44601101,1702997591000


In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, StringType, FloatType, LongType, TimestampType
from pyspark.sql.functions import regexp_replace, concat_ws
from pyspark.sql.functions import when, col, length
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import avg

In [0]:
customers =  spark.read\
.format("parquet")\
.load("dbfs:/mnt/lending_club/staging/customers/non_partitioned/parquet")

In [0]:
customers.show()

+--------------------+--------------------+----------+--------------+-------------+-------------+-----+---------------+---------+-------------------+---------------+----------------+-------------------+-------------------------+---------------+--------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_income|address_state|grade|address_zipcode|sub_grade|verification_status|tot_hi_cred_lim|application_type|joint_annual_income|verification_status_joint|address_country|         ingest_date|
+--------------------+--------------------+----------+--------------+-------------+-------------+-----+---------------+---------+-------------------+---------------+----------------+-------------------+-------------------------+---------------+--------------------+
|b8861edd260ba0394...|             Manager|        10|      MORTGAGE|     102000.0|      USA||KY|401xx|              C|       C2|    Source Verified|       210853.0|      Individual|               NULL|

In [0]:
spark.sql("create database lending_club")

DataFrame[]

In [0]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
|lending_club|
+------------+



In [0]:
spark.sql("use lending_club")

DataFrame[]

In [0]:
customers.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- joint_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- ingest_date: timestamp (nullable = true)



In [0]:
spark.sql("""
CREATE EXTERNAL TABLE lending_club.customers (
    member_id STRING,
    emp_title STRING,
    emp_length INT,
    home_ownership STRING,
    annual_income FLOAT,
    address_state STRING,
    address_zipcode STRING,
    address_country STRING,
    grade STRING,
    sub_grade STRING,
    verification_status STRING,
    total_high_credit_limit FLOAT,
    application_type STRING,
    join_annual_income FLOAT,
    verification_status_joint STRING
)
STORED AS PARQUET
LOCATION 'dbfs:/mnt/lending_club/staging/customers/non_partitioned/parquet'
""")

DataFrame[]

In [0]:
spark.sql("select * from lending_club.customers").show()

+--------------------+--------------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+-----------------------+----------------+------------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_income|address_state|address_zipcode|address_country|grade|sub_grade|verification_status|total_high_credit_limit|application_type|join_annual_income|verification_status_joint|
+--------------------+--------------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+-----------------------+----------------+------------------+-------------------------+
|b8861edd260ba0394...|             Manager|        10|      MORTGAGE|     102000.0|      USA||KY|              C|            USA|401xx|       C2|    Source Verified|                   NULL|      Individual|              NULL|                   

In [0]:
spark.sql("""
create external table if not exists lending_club.loans(
loan_id string, member_id string, loan_amount float, funded_amount float,
loan_term_years integer, interest_rate float, monthly_installment float, issue_date string,
loan_status string, loan_purpose string, loan_title string)
stored as parquet
location 'dbfs:/mnt/lending_club/staging/loans/non_partitioned/parquet'
""")

DataFrame[]

In [0]:
spark.sql("select * from lending_club.loans")

DataFrame[loan_id: string, member_id: string, loan_amount: float, funded_amount: float, loan_term_years: int, interest_rate: float, monthly_installment: float, issue_date: string, loan_status: string, loan_purpose: string, loan_title: string]